# Install Dependencies

In [1]:
!pip install transformers
!pip install lora_adapters
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# Import Modules

In [2]:
import os
import torch
from lora_adapters import LoraConv2d, LoraLinear, LoraEmbedding, apply_adapter, mark_only_lora_as_trainable, lora_state_dict, undo_lora
from torch.optim import AdamW
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, concatenate_datasets
from transformers import TrainingArguments, Trainer, default_data_collator, BertModel
import numpy as np
import evaluate

In [3]:
torch.__version__

'2.0.0'

In [4]:
!nvidia-smi

Sat May  6 03:25:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:23:00.0 Off |                  N/A |
|  0%   40C    P8    23W / 350W |      6MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:2D:00.0  On |                  N/A |
|  0%   

# Load Bert Model

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5).to('cuda')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Adapt Model To Lora

In [6]:
USE_LORA = True
if USE_LORA:
    model = apply_adapter(model, LoraEmbedding, rank=16, regex_pattern=".*word_embeddings")
    model = apply_adapter(model, LoraLinear, rank=16, regex_pattern=".*query")
    model = apply_adapter(model, LoraLinear, rank=16, regex_pattern=".*value")
    model = mark_only_lora_as_trainable(model, bias="lora_only")
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): LoraEmbedding(
        30522, 768, padding_idx=0, rank=16
        (lora_dropout): Identity()
      )
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): LoraLinear(
                in_features=768, out_features=768, bias=True, rank=16
                (lora_dropout): Identity()
              )
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): LoraLinear(
                in_features=768, out_features=768, bias=True, rank=16
                (lora_dropout): Identity()
              

# Train On Downstream Task

In [7]:
dataset = load_dataset("yelp_review_full")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

Found cached dataset yelp_review_full (/home/kian/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(100)).map(tokenize_function, batched=True)
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100)).map(tokenize_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
optimizer = AdamW((param for param in model.parameters() if param.requires_grad), lr=1e-3)

In [10]:
training_args = TrainingArguments(output_dir="test_trainer", per_device_train_batch_size=48, per_device_eval_batch_size=64, report_to="none", bf16=True)

In [11]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

In [13]:
trainer.train()

  0%|          | 0/6 [00:00<?, ?it/s]

/home/kian/anaconda3/envs/torch-2/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'train_runtime': 4.1508, 'train_samples_per_second': 72.275, 'train_steps_per_second': 1.446, 'train_loss': 1.6773816744486492, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=1.6773816744486492, metrics={'train_runtime': 4.1508, 'train_samples_per_second': 72.275, 'train_steps_per_second': 1.446, 'train_loss': 1.6773816744486492, 'epoch': 3.0})

In [14]:
updates = lora_state_dict(model, bias="lora_only")
print(updates.keys())

dict_keys(['bert.embeddings.word_embeddings.weight', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.1.attention.self.query.weight', 'bert.encoder.layer.1.attention.self.value.weight', 'bert.encoder.layer.2.attention.self.query.weight', 'bert.encoder.layer.2.attention.self.value.weight', 'bert.encoder.layer.3.attention.self.query.weight', 'bert.encoder.layer.3.attention.self.value.weight', 'bert.encoder.layer.4.attention.self.query.weight', 'bert.encoder.layer.4.attention.self.value.weight', 'bert.encoder.layer.5.attention.self.query.weight', 'bert.encoder.layer.5.attention.self.value.weight', 'bert.encoder.layer.6.attention.self.query.weight', 'bert.encoder.layer.6.attention.self.value.weight', 'bert.encoder.layer.7.attention.self.query.weight', 'bert.encoder.layer.7.attention.self.value.weight', 'bert.encoder.layer.8.attention.self.query.weight', 'bert.encoder.layer.8.attention.self.value.weight', 'bert.encod

In [24]:
input_tensor = torch.Tensor(small_train_dataset[:10]['input_ids']).to('cuda').long()
output = model(input_tensor)
output.logits.sum().backward()

In [25]:
model_parameters = sum(p.numel() for p in model.parameters())    
model_grads = sum(p.grad.numel() for p in model.parameters() if p.requires_grad)    
optimizer_states = sum([sum(elem.numel() for elem in  p.values()) for p in optimizer.state.values()])

In [26]:
print(f"Model parameters: {model_parameters}")
print(f"Model grads: {model_grads}")
print(f"Optimizer states: {optimizer_states}")

Model parameters: 110576549
Model grads: 1108896
Optimizer states: 2217866


In [27]:
if os.environ.get("COLAB_BACKEND_VERSION", None):
    from google.colab import runtime
    runtime.unassign()